In [1]:
from dotenv import load_dotenv
import os

load_dotenv("./env.local")

from network_analyzer_oo import *

analyzer = NetworkAnalyzer(period=5, half_life=30)

# 10 months of data for initial training
# 2 months of data for validation accuracy
train_start_time = int(datetime(2026, 1, 1).timestamp())
train_end_time = int(datetime(2026, 1, 31).timestamp())
val_start_time = int(datetime(2026, 1, 24).timestamp())
val_end_time = int(datetime(2026, 1, 31).timestamp())


In [5]:
ValDataProcesser = DataProcessor(period=2)
val_dataframe = ValDataProcesser.data_from_api(
    start_time=val_start_time,
    end_time=val_end_time,
    resolution=60
)

ConnectionError: HTTPSConnectionPool(host='tesla-fremont.live.highway9networks.com', port=443): Max retries exceeded with url: /api/v1/orch/devices (Caused by NewConnectionError("HTTPSConnection(host='tesla-fremont.live.highway9networks.com', port=443): Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond"))

In [2]:
analyzer.train_recursive(
    start_time=train_start_time, end_time=train_end_time, resolution=180, interval_days=5
)

Past Data Analysis for Network Connection State
   Found 491 devices

1. Fetching metrics...
   Fetched 491 records
   Processed data shape: (1178891, 9)
   Date range: 2026-01-01 00:00:00 to 2026-01-06 00:00:00

4. Computing probabilities P(state|user,period,weekday)...
Probabilities saved to 'probabilities.json' and counts saved to 'counts.json'

Training from 2026-01-06 00:00:00 to 2026-01-11 00:00:00
   Found 491 devices

1. Fetching metrics...
   Fetched 491 records
Probabilities saved to 'probabilities.json' and counts saved to 'counts.json'

Training from 2026-01-11 00:00:00 to 2026-01-16 00:00:00
   Found 491 devices

1. Fetching metrics...
   Fetched 491 records


KeyboardInterrupt: 

In [1]:
analyzer.load_probabilities()
probability=analyzer.analyzer.get_probabilities()
print(probability[('eud-feed2146-cdc8-4246-95a1-c4f4c18581ff', np.int32(0), np.int32(1))])

NameError: name 'analyzer' is not defined

In [4]:
val_accuracy = analyzer.evaluate(val_dataframe)
print(f"\nValidation Accuracy: {val_accuracy:.2%}")


Validation Accuracy: 33.84%


In [ ]:
class Classifier:
    def __init__(self):
        self._features={} #private attribute to store features
        self.classifications = {}

    @staticmethod
    def entropy(p):
        p = p + 1e-9
        return -np.sum(p * np.log(p))
    
    @staticmethod
    def build_user_tensor(prob_dict):
        """
        Returns:
        user_profiles[user] = array shape (7,96,3)
        """
        user_profiles = defaultdict(lambda: np.zeros((7,720,3)))
        for (user, block, day), probs in prob_dict.items():
            user_profiles[user][day, block, :] = [probs.get(1,0), probs.get(0,0), probs.get(2,0)]

        return user_profiles
    
    def extract_weekly_features(self,user,profile):
        """
        profile shape: (7,96,3)
        """
        feats = {}
        login = profile[:,:,0]   # p_logged_in
        idle  = profile[:,:,2]

        total_login_activity = np.sum(login)
        feats["total_login_activity"] = total_login_activity

        # If device never logs in, stop here
        if total_login_activity < 1e-3:
            feats["inactive"] = True
            self._features[user] = feats
            return feats

        feats["inactive"] = False
        # --- Temporal entropy per day ---
        day_entropies = [self.entropy(login[d]) for d in range(7)]

        feats["login_entropy_mean"] = np.mean(day_entropies)
        feats["login_entropy_var"]  = np.var(day_entropies)

        # --- Weekday consistency ---
        sims = []
        for d1 in range(7):
            for d2 in range(d1+1, 7):
                sims.append(1 - cosine(login[d1], login[d2]))

        feats["weekday_similarity"] = np.mean(sims)

        # --- Peak strength ---
        feats["login_peak_max"] = np.max(login)

        # --- Active days count ---
        feats["active_days"] = np.sum(np.max(login, axis=1) > 0.05)

        # --- Idle dominance ---
        feats["idle_mean"] = np.mean(idle)

        self._features[user] = feats

        return feats

    def classify(self, probabilities: Dict[Tuple, Dict[int, float]]) -> None:
        """
        On the basis of the past data and probabilities classify the users into different categories:
            1. Fixed Wireless Devices
            2. Shift Connected Devices
            3. Ad-hoc Connected Devices (Random Connections)
        """

        user_profiles = self.build_user_tensor(probabilities)
        for user, profile in user_profiles.items():
            feats = self.extract_weekly_features(user, profile)
            if not feats["inactive"] and feats["weekday_similarity"] > 0.85 and feats["login_entropy_mean"] < 2.5:
                classification = "Fixed Wireless Device"
            elif not feats["inactive"] and feats["weekday_similarity"] > 0.6 and feats["active_days"] >= 4:
                classification = "Shift Connected Device"
            else:
                classification = "Ad-hoc Connected Device"
            
            self.classifications[user] = classification

probability = analyzer.analyzer.get_probabilities()
classifier = Classifier()
classifier.classify(probability)
for user, classification in classifier.classifications.items():
    print(f"User: {user}, Classification: {classification}")
    

User: device-02731c1f-17cc-4277-ae8d-e531dc845812, Classification: Ad-hoc Connected Device
User: device-02c645b4-5f24-4e29-87ce-922783cf9e37, Classification: Ad-hoc Connected Device
User: device-02cc17c2-1dc9-47a7-aac6-37704003400f, Classification: Shift Connected Device
User: device-067ad04c-ce71-4244-b6ed-938555d097af, Classification: Shift Connected Device
User: device-0815c5cb-8e5b-4578-ab7f-e21137b672b5, Classification: Shift Connected Device
User: device-08c5a316-94bc-430d-b54c-d4f3fa86e8f0, Classification: Ad-hoc Connected Device
User: device-0c3509f1-415e-45c5-8ee3-ab66514ca8ee, Classification: Ad-hoc Connected Device
User: device-0f3ee68b-764e-4bd7-9639-46fe9f212a47, Classification: Ad-hoc Connected Device
User: device-10578758-6e1b-410f-b111-f2d5e33072ae, Classification: Ad-hoc Connected Device
User: device-10ffe6c0-3f5a-41b4-ad51-364de491d616, Classification: Ad-hoc Connected Device
User: device-11d642a0-2645-4a2b-9824-44dd0a642455, Classification: Ad-hoc Connected Device
Us

c:\Users\naman\Downloads\Device Insights\Device-Insight\.venv\lib\site-packages\scipy\spatial\distance.py:647: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / math.sqrt(uu * vv)


In [ ]:
# TODO: do the predictions on fixed wireless devices, and see if we can improve accuracy near to 100%.

# TODO: Try to generate a cosmatic data on your own. (sample data)

# TODO: add the holidays effect into the model.

# TODO: convert the probability to chart (like heatmap) for better visualization, where horizontal axis is time of day, vertical axis is day of week. connected and disconnected colors, consided idle as connected.

# TODO: resolution should be around 1 minute.

In [ ]:
# Some devices started later in the january, so there predictions won't be good. Look to handel such cases.
